# Preprocesamiento de Datos
---
El preprocesamiento de datos es una etapa crucial en el análisis de datos y aprendizaje automático. Consiste en preparar y limpiar los datos brutos para garantizar su calidad, eliminar inconsistencias, manejar valores faltantes y transformar las variables en formatos adecuados para su análisis o modelado. Esto mejora la precisión y eficiencia de los modelos predictivos.

### Importación de librerías

In [ ]:
import kagglehub
import pandas as pd
import os

### Descarga de dataset desde KaggleHub

In [ ]:
print("Descargando dataset...")
path = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")

### Lectura del archivo de CSV

In [ ]:
data_path = os.path.join(path, "data.csv")
df = pd.read_csv(data_path)

print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
df.head()

In [ ]:
# Información general del dataset
print("Resumen de columnas y tipos de datos:")
df.info()

### Limpieza de Datos

In [ ]:
# Eliminamos filas con valores nulos
df_clean = df.dropna()

print(f"\nDimensiones después de eliminar filas con nulos: {df_clean.shape}")

In [ ]:
# Verificamos balance de la variable objetivo
print("Distribución de la variable objetivo (Bankrupt?):")
print(df_clean["Bankrupt?"].value_counts(normalize=True))

In [ ]:
# Guardamos el dataset limpio para uso posterior
df_clean.to_csv("data_clean.csv", index=False)
print("\nArchivo 'data_clean.csv' guardado correctamente.")